In [7]:
#imports
import requests
import time
import pandas as pd
import json
from bs4 import BeautifulSoup

Creating Urls to gather data from

In [3]:
url_base = 'https://www.reddit.com/r/'
url_tail = '.json'

urls = []
subs = ['smashbros', 'Warhammer40k', 'paintball']

for sub in subs:
    urls.append(url_base + sub + url_tail)

In [3]:
#creating agent name
headers = {'User-agent': 'Some Bot Name'}

Data Collection:

In [4]:
#from Riley

#uses length of urls instead of iterating so that subs list can be
#used to name the csv file
for i in range(len(urls)):
    posts = []
    after = None

    #attempt to get 1000 posts, 40 tries of 25 posts each
    for _ in range(40):
        if after == None:
            params = {}
        else:
            params = {'after': after}
        res = requests.get(urls[i], params=params, headers=headers)
        if res.status_code == 200:
            the_json = res.json()
            posts.extend(the_json['data']['children'])
            after = the_json['data']['after']
        else:
            print(res.status_code)
            break
        time.sleep(1)

    df = pd.DataFrame(posts)

    #formatting data
    df['data'] = df['data'].apply(lambda x: x['title']+': '+x['selftext'])
    df['data'] = df['data'].apply(lambda x: x.replace('\n',''))
    df.drop(columns=['kind'], inplace=True)
    df.to_csv(f'{subs[i]}.csv', index=False)